## Agentic RAG
최신 정보에 접근하여 검색결과를 가지고 답변을 생성하는 에이전트.<br>
질문에 따라 문서를 검색하여 답변하거나, 인터넷 검색 도구를 활용하여 답변하는 에이전트.<br>
**RAG 를 수행하되, Agent를 활용하여 RAG를 수행한다면 이를 Agentic RAG라고 함.**

## 도구(Tools)
Agent가 활용할 도구를 정의하여 Agent가 추론(reasoning)을 수행할 때 활용하도록 할 수 있다.<br>

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith('Agents')

LangSmith 추적을 시작합니다.
[프로젝트명]
Agents


In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

# TavilySearchResults 클래스의 인스턴스를 생성합니다
# k=6은 검색 결과를 6개까지 가져오겠다는 의미입니다
search = TavilySearchResults(k=6)

In [4]:
search.invoke("판교 카카오 프렌즈샵 아지트점의 전화번호는 무엇인가요?")

[{'url': 'https://blog.naver.com/PostView.naver?blogId=amour1115&logNo=223386213560',
  'content': '카카오 프렌즈 판교 아지트점. . 연중무휴 12:00-21:00. 토, 일 12:00-20:00. . 경기 성남시 분당구 판교역로 166. 카카오 판교 아지트 1층 카카오 프렌즈. 031-301-7225. 존재하지 않는 이미지입니다.'},
 {'url': 'https://blog.naver.com/PostView.naver?blogId=twsilver0&logNo=223614680235',
  'content': '판교 카카오 아지트 :: 귀여움 가득한 판교역 카카오 프렌즈샵 방문기 건물이 너무 화려해서 완전 제 취향이였네요 카카오 아지트 한번 구경해줘야죠 카카오 아지트 입성 카카오 아지트 구경을 해줘요 곳곳에 카카오 프렌즈 캐릭터가 있어서 카카오 아지트 1층에 카카오 프렌즈 판교아지트점 죠르디 랜덤 미니 키링 인형 카카오 아지트 들리실 일 있으면  카카오 프렌즈 샵은 꼭 가주세요  카카오 아지트 구경해보는거 추천 드립니다 고객님의 PC가 악성코드에 감염될 경우 시스템성능 저하,개인정보 유출등의 피해를 입을 수 있으니 주의하시기 바랍니다. 저작권을 침해하는 컨텐츠가 포함되어 있는 게시물의 경우 글보내기 기능을 제한하고 있습니다. 건강한 인터넷 환경을 만들어 나갈 수 있도록 고객님의 많은 관심과 협조를 부탁드립니다. 저작권을 침해하는 컨텐츠가 포함되어 있는 게시물의 경우 주제 분류 기능을 제한하고 있습니다. 건강한 인터넷 환경을 만들어 나갈 수 있도록 고객님의 많은 관심과 협조를 부탁드립니다.'},
 {'url': 'https://m.blog.naver.com/kimsso333/223186761448',
  'content': '\u200b\n\u200b\n\u200b\n\u200b\n선물 후보로 끝까지 고민했던\n춘식이 얼음틀 ㅋㅋㅋ\n\u200b\n\u200b\n\u200b\n\u2

## 문서 기반 문서 검색 도구: Retrieval


In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드, 파일 경로 입력
loader = PyPDFLoader('data/SPRI_AI_Brief_2023년12월호_F.pdf')

# 텍스트 분할기를 통해 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

# 문서 로드 분할
split_docs = loader.load_and_split(text_splitter)

# 벡터 저장소 생성
vector_store = FAISS.from_documents(split_docs, OpenAIEmbeddings())

retriever = vector_store.as_retriever()


In [6]:
# 문서에서 관련성 높은 문서 찾기.
retriever.invoke("삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘")

[Document(metadata={'source': 'data/SPRI_AI_Brief_2023년12월호_F.pdf', 'page': 12}, page_content='SPRi AI Brief |  \n2023-12 월호\n10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개\nn삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 \nAI 모델 ‘삼성 가우스 ’를 공개\nn삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 \n삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents\n£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원\nn삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 \n‘삼성 가우스 ’를 최초 공개\n∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 \n최적화된 크기의 모델 선택이 가능\n∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , \n온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유\n∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 \n단계적으로 탑재할 계획\nn삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  \n이미지 모델의 3개 모델로 구성\n∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 \n처리를 지원\n∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 인터페이스로 서비스를 제공하며 \n사내 소프트웨어 개발에 최적화\n∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원하는 대로 바꿀 수 있

에이전트가 사용할 수 있는 도구(Tool)로 변환.
create_retriever_tool 함수로 retriever를 도구로 변환.

In [9]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    name = 'pdf_search',
    description = 'use this tool to search for information in the PDF document'
)

Agent가 사용할 도구 목록 정의


In [10]:
tools = [search, retriever_tool]

## Agent 생성

Agent가 활용할 LLM 정의, 프롬프트 정의.
- 멀티턴 대화를 지원하지 않을거라면 'chat_history' 제거해도 무방

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature = 0.0
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant,"
    "Make sure to use the 'pdf_search' tool for searching information in the PDF document."
    "If you can't find the information from the PDF document, use the 'search' tool for searching information from the web."
    ),
    ("placeholder", "{chat_history}"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [12]:
# Tool calling Agent 생성
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)


생성한 agent를 실행하는 AgentExecutor 생성

In [14]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

## Agent 실행

In [15]:
from langchain_teddynote.messages import AgentStreamParser

agent_stream_parser = AgentStreamParser()

In [17]:
result = agent_executor.stream(
    {"input": "2024년 프로야구 한국시리즈 우승 팀이 누구인지 알려줘."}
)
for step in result:
    agent_stream_parser.process_agent_steps(step)


[도구 호출]
Tool: tavily_search_results_json
query: 2024년 프로야구 한국시리즈 우승 팀
Log: 
Invoking: `tavily_search_results_json` with `{'query': '2024년 프로야구 한국시리즈 우승 팀'}`



[관찰 내용]
Observation: [{'url': 'https://namu.wiki/w/2024+신한+SOL+Bank+KBO+한국시리즈', 'content': '2024년 kbo 한국시리즈 우승팀. ... 올 시즌 kia는 한국프로야구 43년 역사상 단일 시즌 최다 실책을 기록했을 만큼 매우 불안한 수비를 보여주고 있다. ... 가을야구 경험 부족 2017년 한국시리즈 이후 포스트시즌이 단 2번 뿐이고, 그마저도 전부 5위로 진출해서 와일드'}, {'url': 'https://m.blog.naver.com/dicasub/223637908357', 'content': '프로야구 한국시리즈 역대 우승팀 및 팀별 우승 횟수(~2024년까지) : 네이버 블로그 본문 바로가기 카테고리 이동 디카섭(DiCaSub)의 스포츠 통계 이야기 야구[국내일반] [공지] 프로야구 한국시리즈 역대 우승팀 및 팀별 우승 횟수(~2024년까지) 디카섭 DiCaSub 이웃추가 본문 기타 기능 공유하기 국내 프로야구(KBO) 한국시리즈 역대 우승팀 정리 삼성과의 상대전적 4승 1패로 KIA가 한국시리즈 우승 트로피를 들어올렸습니다!! 역대 한국시리즈 우승팀 현황을 아래 <표>와 <그림>으로 정리해보았습니다. 국내 프로야구(KBO) 구단(팀)별 한국시리즈 우승 횟수<표> 국내 프로야구(KBO) 구단(팀)별 한국시리즈 우승 횟수<그림> [그림1] 국내 프로야구(KBO) 구단(팀)별 한국시리즈 우승 횟수 (막대형) [그림2] 국내 프로야구(KBO) 구단(팀)별 한국시리즈 우승 횟수 (원형) https://blog.naver.com/dicasub/223603577717 2024년 국내 프로야구(KBO) 정규리그 

In [18]:
result = agent_executor.stream(
    {"input": "삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘"}
)
for step in result:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원

## 이전 대화내용 기억하는 Agent
이전의 대화내용을 기억하기 위해서는 RunnableWithMessageHistory 를 사용하여 AgentExecutor를 감싸줌.


In [20]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory

# session_id를 저장할 딕셔너리
store = {}

def get_session_history(session_id):
    if session_id not in store:
        # 세션 아이디가 없으면 새로운 ChatMessageHistory 인스턴스 생성
        store[session_id] = ChatMessageHistory()
    return store[session_id]

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # 대화 session_id
    get_session_history,
    # 프롬프트 질문이 입력되는 key: input
    input_messages_key="input",
    # 프롬프트의 메세지가 입력되는 key: chat_history
    history_messages_key="chat_history",
)


In [21]:
response = agent_with_chat_history.stream(
    {"input": "삼성전자가 개발한 생성형 AI 관련 내용을 문서에서 찾아줘"},
    # session_id 설정
    config = {"configurable": {"session_id": "123"}}

)
for step in response:
    agent_stream_parser.process_agent_steps(step)

[도구 호출]
Tool: pdf_search
query: 삼성전자 생성형 AI
Log: 
Invoking: `pdf_search` with `{'query': '삼성전자 생성형 AI'}`



[관찰 내용]
Observation: SPRi AI Brief |  
2023-12 월호
10삼성전자 , 자체 개발 생성 AI ‘삼성 가우스 ’ 공개
n삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스 ’를 공개
n삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로 , 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유KEY Contents
£언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스 , 온디바이스 작동 지원
n삼성전자가 2023 년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스 ’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스 (Gauss) 의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며 , 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며 , 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는  
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며 , 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이 (code.i)’ 는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 모델은 창의적인 이미지를 생성하고 기존 이미지를 원

In [22]:
response = agent_with_chat_history.stream(
    {"input": "이전의 답변 영어로 번역해줘"},
    # session_id 설정
    config = {"configurable": {"session_id": "123"}}

)
for step in response:
    agent_stream_parser.process_agent_steps(step)

[최종 답변]
Here is the translation of the previous response into English:

- **Samsung Gauss**: On November 8, 2023, Samsung Electronics unveiled its generative AI model called 'Samsung Gauss' at the 'Samsung AI Forum 2023'. This model consists of three components: language, code, and image models, and is designed to operate on-device, ensuring that user information is not at risk of being leaked externally.

- **Model Composition**: Samsung Gauss is composed of the following three models:
  1. **Language Model**: Supports tasks such as email writing, document summarization, and translation.
  2. **Code Model**: Provides a conversational interface through the AI coding assistant 'code.i', optimized for in-house software development.
  3. **Image Model**: Capable of generating creative images and modifying existing ones, as well as converting low-resolution images to high-resolution.

- **Safe Data Learning**: Samsung Gauss is trained on safe data that does not infringe on licenses or pers